In [ ]:
# general
from datetime import datetime
from dotenv import load_dotenv, find_dotenv
import os

# custom local libs
from function import baseball_lambda

In [ ]:
# read local .env file
_ = load_dotenv(find_dotenv()) 

In [ ]:
# set Langsmith project
today = datetime.now().strftime("%Y%m%d")
os.environ["LANGCHAIN_PROJECT"] = f"Baseball Curveballs A - {today}"

In [ ]:
session_id = '291114'

In [ ]:
task =  "Consider the first week of August 2020 - find 3 pitchers who's curveballs were most similar to Max Scherzer's."
result = baseball_lambda.execute_workflow(task, session_id)
print(result)

In [ ]:
task = """
make sure the plan follows this general flow:
1) create an average vector for each pitcher's curveball.
2) Train a knn model on this data
3) Use this model to find the 3 pitchers with the most similar curveball
"""
result = baseball_lambda.execute_workflow(task, session_id)
print(result)

In [ ]:
task = 'approved'
result = baseball_lambda.execute_workflow(task, session_id)
print(result)

In [ ]:
task = 'yes'
result = baseball_lambda.execute_workflow(task, session_id)
print(result)

In [ ]:
task = 'yes, thank you'
result = baseball_lambda.execute_workflow(task, session_id)
print(result)